In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
import os
from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
# import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [2]:
# load Mask_ERF_files
Mask_ERF = iris.load('../saved-files/threshold-15/Mask_ERF_iris-15.nc')[0]
Mask_ERF

<iris 'Cube' of segmentation_mask / (1) (time: 301; -- : 400; -- : 400)>

In [3]:
track = pd.read_csv('../saved-files/threshold-15/final-tracks-threshold-15-excluding-area.csv')
track.head()

frame  idx      hdim_1      hdim_2  num  threshold_value  feature  \
0      0    2  133.891782  247.588144    6               15        2   
1      1    5  135.850880  248.149120    2               15       86   
2      2    5  139.000000  249.440269    2               15      169   
3      3    4  143.476756  250.870342    6               15      251   
4      4    5  148.000000  253.920901    3               15      336   

                  time              timestr  projection_y_coordinate  \
0  2020-03-13 04:00:00  2020-03-13 04:00:00            -2.193892e+06   
1  2020-03-13 04:05:00  2020-03-13 04:05:00            -2.195851e+06   
2  2020-03-13 04:10:00  2020-03-13 04:10:00            -2.199000e+06   
3  2020-03-13 04:15:00  2020-03-13 04:15:00            -2.203477e+06   
4  2020-03-13 04:20:00  2020-03-13 04:20:00            -2.208000e+06   

   projection_x_coordinate   latitude  longitude  cell        time_cell  \
0            164588.143590  70.196696  14.290355     2  0 days 00:00:00   
1            165149.120397  70.178537  14.301097     2  0 days 00:05:00   
2            166440.268555  70.149081  14.328411     2  0 days 00:10:00   
3            167870.342039  70.107474  14.356623     2  0 days 00:15:00   
4            170920.901252  70.064303  14.426429     2  0 days 00:20:00   

     distance   mean dBZ    max dBZ  
0  125.209617  13.245867  34.567482  
1  123.291260  12.974534  35.577388  
2  119.982671  12.780750  34.063904  
3  115.540119  12.708913  36.584485  
4  110.078668  12.845599  38.092506

In [4]:
len(track)

17805

## _CALCULATE AREA_

In [6]:
area = tobac.calculate_area(track, Mask_ERF, method_area=None)
# area1 = (c['area']/1000000)
area1 = np.divide(area['area'],1000000)
area['area($km^2$)'] = area1

In [7]:
track['area($km^2$)'].max()

2574.057090682289

In [8]:
# # finding the maximum area
# z=track[track['area($km^2$)'] == 1433.9665540745268]
# z

In [9]:
tracks = track.drop(columns=['area'])
tracks

frame  idx      hdim_1      hdim_2  num  threshold_value  feature  \
0          0    2  133.891782  247.588144    6               15        2   
1          1    5  135.850880  248.149120    2               15       86   
2          2    5  139.000000  249.440269    2               15      169   
3          3    4  143.476756  250.870342    6               15      251   
4          4    5  148.000000  253.920901    3               15      336   
...      ...  ...         ...         ...  ...              ...      ...   
17800    299    4  144.743717  239.312025    4               15    23004   
17801    300    7  146.461260  240.148842    5               15    23041   
17802    298   31  256.000000  174.366873    2               15    22993   
17803    299   27  256.815629  176.111734    4               15    23027   
17804    300   27  258.000000  178.000000    1               15    23061   

                      time              timestr  projection_y_coordinate  \
0      2020-03-13 04:00:00  2020-03-13 04:00:00            -2.193892e+06   
1      2020-03-13 04:05:00  2020-03-13 04:05:00            -2.195851e+06   
2      2020-03-13 04:10:00  2020-03-13 04:10:00            -2.199000e+06   
3      2020-03-13 04:15:00  2020-03-13 04:15:00            -2.203477e+06   
4      2020-03-13 04:20:00  2020-03-13 04:20:00            -2.208000e+06   
...                    ...                  ...                      ...   
17800  2020-03-14 04:55:00  2020-03-14 04:55:00            -2.204744e+06   
17801  2020-03-14 05:00:00  2020-03-14 05:00:00            -2.206461e+06   
17802  2020-03-14 04:50:00  2020-03-14 04:50:00            -2.316000e+06   
17803  2020-03-14 04:55:00  2020-03-14 04:55:00            -2.316816e+06   
17804  2020-03-14 05:00:00  2020-03-14 05:00:00            -2.318000e+06   

       projection_x_coordinate   latitude  longitude  cell        time_cell  \
0                164588.143590  70.196696  14.290355     2  0 days 00:00:00   
1                165149.120397  70.178537  14.301097     2  0 days 00:05:00   
2                166440.268555  70.149081  14.328411     2  0 days 00:10:00   
3                167870.342039  70.107474  14.356623     2  0 days 00:15:00   
4                170920.901252  70.064303  14.426429     2  0 days 00:20:00   
...                        ...        ...        ...   ...              ...   
17800            156312.025471  70.103689  14.055373  6578  0 days 00:05:00   
17801            157148.841602  70.087558  14.073848  6578  0 days 00:10:00   
17802             91366.873305  69.124388  12.259164  6586  0 days 00:00:00   
17803             93111.734429  69.116329  12.301452  6586  0 days 00:05:00   
17804             95000.000000  69.104843  12.346874  6586  0 days 00:10:00   

         distance   mean dBZ    max dBZ  area($km^2$)  
0      125.209617  13.245867  34.567482      8.952468  
1      123.291260  12.974534  35.577388     20.890860  
2      119.982671  12.780750  34.063904     21.887120  
3      115.540119  12.708913  36.584485     22.880821  
4      110.078668  12.845599  38.092506           NaN  
...           ...        ...        ...           ...  
17800  122.013418  15.240493  41.116443           NaN  
17801  120.179000  14.702386  40.106802      5.970119  
17802  148.655096  15.301029  39.603226     10.944407  
17803  147.098363  15.240493  41.116443     10.943279  
17804  145.475724  14.702386  40.106802      9.950738  

[17805 rows x 19 columns]

In [10]:
# all tracks
tracks.to_csv('../saved-files/threshold-15/final-tracks-threshold-15.csv', index=False)

## _PROOF THAT WE HAVE AREA OF THIS KIND_

In [11]:
# mask_cell_361 = tobac.utils.mask_cell(Mask_ERF, 361, tracks)
# mask_cell_361

In [12]:
# convert = xr.DataArray.from_iris(mask_cell_361)
# convert

In [13]:
# df_361 = tracks[tracks['cell'] == 361]
# df_361

In [14]:
# %%time
# path = '/glade/u/home/noteng/work/research/data/'
# file = 'yrwms-nordic.mos.pcappi-0-dbz.noclass-clfilter-novpr-clcorr-block.laea-yrwms-1000.20200313.nc'
# data = xr.open_dataset(path+file)
# # equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,450:580,256:771] #Based on longitude and latitude of Andoya and Norwegian Sea
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,250:650,450:850] #Based on longitude and latitude of Andoya and Norwegian Sea
# # equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,330:580,660:780] #### hdm1 and hdm2
# # equivalent_reflectivity_factor = data['equivalent_reflectivity_factor']
# equivalent_reflectivity_factor
# data.close()

In [15]:
# %%time

# for i in range(0, 9+1):

#     ## center points and extent of map ###
#     latA = 69.141281 #latitude of COMBLE site
#     lonA = 15.684166-1 #longitude of COMBLE site -1
#     xm, ym = 6,2.75 # extent from center point in lon and lat

#     fig, ax = plt.subplots(1,1,figsize=(10,9),subplot_kw={'projection': ccrs.Orthographic(lonA,latA)})

#     extent = (lonA-xm, lonA+xm, latA-ym, latA+ym) # define extent map
#     ax.set_extent(extent) # set extent of map
#     ax.coastlines(resolution='10m') # plot coastlines with high resolution: 10m


#     #change time to moth, day, year, hours and minutes
#     # dt = masked_values_to_xarray[4]['time'].dt.strftime('%m-%d-%Y %H:%M').values  ### array 
#     dt = convert['time'].dt.strftime('%m-%d-%Y %H:%M').values  ### array 
    

# #     #     # fill in the place where there is cloud with 1 and place with no cloud to be zero
#     cloud = np.asarray(convert[i],dtype='float')
#     invalid_cloud = np.greater(cloud,0)
#     cloud[invalid_cloud] = 1
#     invalid_cloud1 = np.less(cloud,1)
#     cloud[invalid_cloud1] = np.nan
    

#     cs=ax.pcolormesh(convert['lon'],
#                 convert['lat'], 
#                 # cloud*equivalent_reflectivity_factor.values[i],
#                 cloud*equivalent_reflectivity_factor.values[i],
#                 cmap='jet',
#                 transform=ccrs.PlateCarree(),
#                 vmin=-20,
#                 vmax=40
#                 )

#     ax.plot(lonA+1,latA, color='red', marker='*', markersize = 10.5,transform=ccrs.PlateCarree()) # plot red star at location of Andenes
#     # ax.plot(lonA,latA, color='red', marker='*', markersize = 20.5,transform=ccrs.PlateCarree()) # plot red star at location of Andenes
#     ax.set_title(f"Nordic Radar Mosaic: {dt[i]} UTC", fontweight='bold', fontsize=16, pad=11)  # title of figure


#     # Add a colorbar axis at the right of the graph
#     cbar_ax = fig.add_axes([0.85, 0.13, 0.047, 0.75]) #lbwh


#     cbar=fig.colorbar(cs, cax=cbar_ax, orientation='vertical')
#     cbar.set_label("Reflectivity Factor [dBZ]", fontsize = 15)
    
#     fig.savefig(f'./new//{dt[i]}.png', dpi=100)
#     plt.close()



# # #     dirr = sorted(os.listdir('./mod-images//'))
# # #     if f'{dt[i]}.png' in dirr:
# # #         # print(f'{dt[i]}.png already exist')
# # #         pass
# # #         plt.close()
# # #     else:
# # #         fig.savefig(f'./mod-images//{dt[i]}.png', dpi=100)
# # #         plt.close()
# # print('\U0001f600\U0001f600\U0001f600\U0001f600EXECUTION DONE!!!\U0001f600\U0001f600\U0001f600\U0001f600')
# # gc.collect()

In [16]:
# %%time
# import glob
# # Build GIF
# imgs = sorted(glob.glob("./new/*.png"))
# with imageio.get_writer('large-area.gif', mode='I', duration=0.3) as writer:
#     for filename in imgs:
#         image = imageio.v2.imread(filename)
#         writer.append_data(image)

In [17]:
# %%time
# #Display image
# display.Image('large-area.gif', width=800)

# END 